# Implimentation of the 4. SPML Assignment
## Taxi Environment
## Table of Content
* Libraries
* Environment
* Value Iteration
    * Algorithm
    * Training V and Policy
    * Graphical Representation
    
    
* Q-Learning
    * Algorithm
    * Training Q-Values
    * Graphical Representation

## Libraries

In [1]:
import numpy as np # Linear Algebra
import gym         # Agent's Environment
import random      # Random values
import time        # Timesteps

## Environment

* env.reset()
    * resets the environment
    * returns a random initial state
* env.step(action)
    * one time-step in environment
    * returns
        * observation
            * environmental observations
        * reward
            * was the action beneficial
        * done
            * did the agant successfully pick up and drop passangers
            * (one episode)
        * info
            * for debugging
* env.render()
    * renders one frame of the environment
    * visual purpose

In [2]:
env = gym.make('Taxi-v2').env

In [3]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Value Iteration

### Algorithm

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://cdn-images-1.medium.com/max/1600/1*MsD6og8hCReDO24T8iZfNw.png")

1. Initialize V(s) and V(s') to different arbitary values. Otherwise the while loop below would not execute
2. Initialize a policy arbitary where the row-size euqals the #states & the column-size equals the #actions
3. Repeat 4-9 till difference of V(s) and V(s') is smaller than some small specific amount
4. V(s) updates to be V(s')
5. Go through all states and in every state look which action are available 
6. Make a list where all the q-values for the possible state, action pairs will be put in
7. Put the q-values according to the bellman equations into the list created in step 4
8. Update V(s') to be the maximum value from the q-value list
9. One-hot-encoding the the action we take and reasing solution to policy array

#### The Bellman Equation

In [5]:
def bellman_equation(prob_nextState_reward_goal, gamma, V_old):
    prob_to_nextState = prob_nextState_reward_goal[0][0]
    nextState = prob_nextState_reward_goal[0][1]
    reward = prob_nextState_reward_goal[0][2]
    return reward + gamma*(prob_to_nextState*V_old[nextState])

### Training V and Policy

In [6]:
def value_iteration(env, theta=0.0001, discount_factor=0.9):
    # 1. Initialize V(s) and V(s') to different arbitary values
    V_new = np.zeros(env.nS)
    V_old = np.ones(env.nS)
    # 2. Initialize a policy arbitary where the row-size euqals the #states 
    # & the column-size equals the #actions
    policy = np.zeros([env.nS, env.nA])
    
    # 3. Repeat 4-7 till difference of V(s) and V(s') is smaller
    # than some small specific amount
    while np.all(np.abs(V_new-V_old) > theta):
        # 4. V(s) updates to be V(s')
        V_old = np.copy(V_new)
        # 5. Go through all states and in every state look 
        # which action are available 
        for state in range(env.nS):
            # 6. Make a list where all the q-values for 
            # the possible actions are put in
            q_s_a = []   # Q(state, action)
            for action in range(env.nA):
                # 7. Put the q-values according to the bellman 
                # equations into the list created in step 4
                prob_nextState_reward_goal = env.P[state][action]
                q_s_a.append( bellman_equation(prob_nextState_reward_goal, discount_factor, V_old) )
            # 8. Update V(s') to be the maximum value from the q-value list
            V_new[state] = max(q_s_a)
            # 9. One-hot-encoding the the action we take and 
            # reasing solution to policy array
            policy[state]=0
            policy[state, np.argmax(q_s_a)] = 1
    return policy, V_new

In [7]:
policy, V = value_iteration(env)

In [8]:
policy

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [9]:
V

array([178.99911406,  90.44146306, 114.00201406,  80.39722816,
        43.37829646,  90.44146306,  33.2362518 ,  49.30931672,
        71.35741675,  43.37829646, 114.00201406,  49.30931672,
        33.2362518 ,  43.37829646,  33.2362518 ,  80.39722816,
       199.99911406, 101.60172406, 127.78011406,  90.44146306,
       160.09911406,  80.39722816, 101.60172406,  71.35741675,
        49.30931672, 101.60172406,  38.04037822,  55.89933924,
        63.22158648,  38.04037822, 101.60172406,  43.37829646,
        38.04037822,  49.30931672,  38.04037822,  90.44146306,
       178.99911406, 114.00201406, 114.00201406, 101.60172406,
       114.00201406,  55.89933924,  71.35741675,  49.30931672,
        71.35741675, 143.08911406,  55.89933924,  80.39722816,
        55.89933924,  33.2362518 ,  90.44146306,  38.04037822,
        43.37829646,  55.89933924,  43.37829646, 101.60172406,
       127.78011406, 160.09911406, 101.60172406, 114.00201406,
       101.60172406,  49.30931672,  63.22158648,  43.37

In [10]:
policy.shape, V.shape

((500, 6), (500,))

### Graphical Representation

In [9]:
state = env.reset()
done = False
while not done:
    action = np.argmax(policy[state])
    print('Action: ', action)
    next_state, reward, done, info = env.step(action)
    env.render()
    time.sleep(1)
    state = next_state

0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
3
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
4
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
1
+---------+
|R: | : :G|
| : : : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (North)
3
+---------+
|R: | : :G|
| : : : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
3
+---------+
|R: | : :G|
| : : : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
3
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (South)
0
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
5
+---------+
|R: | : :G|
| : : : : |
| : :

## Q-Learning

### Algorithm

In [10]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://cdn-images-1.medium.com/max/1600/1*_CRZzhrq1KDI2yKYq5AREg.png")

1. Initialize Q-Table
    * random
    * or all zeros
    * or domain knowledge initialization
2. Explore vs Exploit Action
    * select an action for current state s
3. Execute selected Action -> Go to s'
4. Selcet Action in s'
    * action with highest q-value
5. Update Q-Table according to the Bellman Equations
    * $Q({\small state}, {\small action}) \leftarrow (1 - \alpha) Q({\small state}, {\small action}) + \alpha \Big({\small reward} + \gamma \max_{a} Q({\small next \ state}, {\small all \ actions})\Big)$
6. Update the Current State to be the Next State
7. If Goal-State -> New Episode

### Training Q-Values

In [11]:
# Values we need before we start with the actual Algorithm
state_space = env.observation_space.n
action_space = env.action_space.n
state_space, action_space

(500, 6)

In [12]:
#1. Initialize Q-Table
q_table = np.zeros([state_space, action_space])
q_table.shape

(500, 6)

In [13]:
def q_learning(env, q_table, epsilon=0.1, alpha=0.1, gamma=0.6):
    for episode in range(1, 100001):
        state = env.reset()
        done = False
        # 7. If Goal-State -> New Episode
        while not done:
            # 2. Explore vs Exploit Action
            if random.uniform(0, 1) < epsilon:
                action = random.randint(0, 5)
            else:
                action = np.argmax(q_table[state])
            
            # 3. Execute selected action -> Go to s'
            next_state, reward, done, info = env.step(action)
            
            # 4. Update Q-Table according to the Bellman Equations
            max_next_action = np.max(q_table[next_state])
            q_table[state][action] = q_table[state][action] + alpha*(reward + gamma*max_next_action-q_table[state][action])
            # OR 
            # q_table[state][action] = (1-alpha)*q_table[state][action] + alpha*(reward+gamma*max_next_action)
            
            # Update the Current State to be the Next State
            state = next_state
        
        if episode%100 == 0:
            print(str(episode) + ' Episode')

In [14]:
q_learning(env, q_table)

100 Episode
200 Episode
300 Episode
400 Episode
500 Episode
600 Episode
700 Episode
800 Episode
900 Episode
1000 Episode
1100 Episode
1200 Episode
1300 Episode
1400 Episode
1500 Episode
1600 Episode
1700 Episode
1800 Episode
1900 Episode
2000 Episode
2100 Episode
2200 Episode
2300 Episode
2400 Episode
2500 Episode
2600 Episode
2700 Episode
2800 Episode
2900 Episode
3000 Episode
3100 Episode
3200 Episode
3300 Episode
3400 Episode
3500 Episode
3600 Episode
3700 Episode
3800 Episode
3900 Episode
4000 Episode
4100 Episode
4200 Episode
4300 Episode
4400 Episode
4500 Episode
4600 Episode
4700 Episode
4800 Episode
4900 Episode
5000 Episode
5100 Episode
5200 Episode
5300 Episode
5400 Episode
5500 Episode
5600 Episode
5700 Episode
5800 Episode
5900 Episode
6000 Episode
6100 Episode
6200 Episode
6300 Episode
6400 Episode
6500 Episode
6600 Episode
6700 Episode
6800 Episode
6900 Episode
7000 Episode
7100 Episode
7200 Episode
7300 Episode
7400 Episode
7500 Episode
7600 Episode
7700 Episode
7800 Epi

59500 Episode
59600 Episode
59700 Episode
59800 Episode
59900 Episode
60000 Episode
60100 Episode
60200 Episode
60300 Episode
60400 Episode
60500 Episode
60600 Episode
60700 Episode
60800 Episode
60900 Episode
61000 Episode
61100 Episode
61200 Episode
61300 Episode
61400 Episode
61500 Episode
61600 Episode
61700 Episode
61800 Episode
61900 Episode
62000 Episode
62100 Episode
62200 Episode
62300 Episode
62400 Episode
62500 Episode
62600 Episode
62700 Episode
62800 Episode
62900 Episode
63000 Episode
63100 Episode
63200 Episode
63300 Episode
63400 Episode
63500 Episode
63600 Episode
63700 Episode
63800 Episode
63900 Episode
64000 Episode
64100 Episode
64200 Episode
64300 Episode
64400 Episode
64500 Episode
64600 Episode
64700 Episode
64800 Episode
64900 Episode
65000 Episode
65100 Episode
65200 Episode
65300 Episode
65400 Episode
65500 Episode
65600 Episode
65700 Episode
65800 Episode
65900 Episode
66000 Episode
66100 Episode
66200 Episode
66300 Episode
66400 Episode
66500 Episode
66600 

### Graphical Representation

In [15]:
state = env.reset()
done = False
while not done:
    action = np.argmax(q_table[state])
    next_state, reward, done, info = env.step(action)
    env.render()
    time.sleep(1)
    state = next_state

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | :_:G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : :_: : |
| : : : : |
| | : | : |
|Y|